In [37]:
import json
with open('/Users/justindevine/.secret/Data/yelp_api.json') as f:
    login = json.load(f)

In [38]:
# Standard Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# Additional Imports
import os, json, math, time
from yelpapi import YelpAPI
from tqdm.notebook import tqdm_notebook

In [39]:
yelp_api = YelpAPI(login['api-key'], timeout_s=5.0)

In [40]:
# set our API call parameters 
LOCATION = 'New Orleans'
TERM = 'Gumbo'

In [41]:
# Specifying JSON_FILE filename (can include a folder)
# include the search terms in the filename
JSON_FILE = "Data/results_NO_Gumbo.json"
JSON_FILE

'Data/results_NO_Gumbo.json'

In [42]:
## Check if JSON_FILE exists
file_exists = os.path.isfile(JSON_FILE)
## If it does not exist: 
if file_exists == False:
    
    ## CREATE ANY NEEDED FOLDERS
    # Get the Folder Name only
    folder = os.path.dirname(JSON_FILE)
    ## If JSON_FILE included a folder:
    if len(folder)>0:
        # create the folder
        os.makedirs(folder,exist_ok=True)
        
        
    ## INFORM USER AND SAVE EMPTY LIST
    print(f'[i] {JSON_FILE} not found. Saving empty list to file.')
    
    
    # save an empty list
    with open(JSON_FILE,'w') as f:
        json.dump([],f)  
# If it exists, inform user
else:
    print(f"[i] {JSON_FILE} already exists.")

[i] Data/results_NO_Gumbo.json not found. Saving empty list to file.


In [43]:
## Load previous results and use len of results for offset
with open(JSON_FILE,'r') as f:
    previous_results = json.load(f)
    
## set offset based on previous results
n_results = len(previous_results)
print(f'- {n_results} previous results found.')

- 0 previous results found.


In [44]:
# use our yelp_api variable's search_query method to perform our API call
results = yelp_api.search_query(location=LOCATION,
                                term=TERM,
                               offset=n_results)
results.keys()

dict_keys(['businesses', 'total', 'region'])

In [45]:
## How many results total?
total_results = results['total']
total_results

555

In [46]:
## How many did we get the details for?
results_per_page = len(results['businesses'])
results_per_page

20

In [47]:
# Import additional packages for controlling our loop
import time, math
# Use math.ceil to round up for the total number of pages of results.
n_pages = math.ceil((results['total']-n_results)/ results_per_page)
n_pages

28

In [48]:
# join new results with old list with extend and save to file
previous_results.extend(results['businesses'])  
with open(JSON_FILE,'w') as f:
     json.dump(previous_results,f)

In [49]:
from tqdm.notebook import tqdm_notebook
import time
for i in tqdm_notebook(range(n_pages)):
    # adds 200 ms pause
    time.sleep(.2) 

  0%|          | 0/28 [00:00<?, ?it/s]

In [50]:
for i in tqdm_notebook( range(1,n_pages+1)):
    
    ## Read in results in progress file and check the length
    with open(JSON_FILE, 'r') as f:
        previous_results = json.load(f)
    ## save number of results for to use as offset
    n_results = len(previous_results)
    ## use n_results as the OFFSET 
    results = yelp_api.search_query(location=LOCATION,
                                    term=TERM, 
                                    offset=n_results)
    
    ## append new results and save to file
    previous_results.extend(results['businesses'])
    
    with open(JSON_FILE,'w') as f:
        json.dump(previous_results,f)
    
    # add a 200ms pause
    time.sleep(.2)

  0%|          | 0/28 [00:00<?, ?it/s]

In [51]:
# load final results
final_df = pd.read_json(JSON_FILE)
display(final_df.head(), final_df.tail())

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,qb28j-FNX1_6xm7u372TZA,gumbo-shop-new-orleans-2,Gumbo Shop,https://s3-media2.fl.yelpcdn.com/bphoto/s3LSJn...,False,https://www.yelp.com/biz/gumbo-shop-new-orlean...,4524,"[{'alias': 'cajun', 'title': 'Cajun/Creole'}]",4.0,"{'latitude': 29.957692211278957, 'longitude': ...",[delivery],$$,"{'address1': '630 Saint Peter St', 'address2':...",+15045251486,(504) 525-1486,1862.914864
1,SJIQFKTW6uUsOo29w9IHxw,mambos-new-orleans,Mambo's,https://s3-media4.fl.yelpcdn.com/bphoto/tarz6Y...,False,https://www.yelp.com/biz/mambos-new-orleans?ad...,2162,"[{'alias': 'cajun', 'title': 'Cajun/Creole'}, ...",4.0,"{'latitude': 29.95668, 'longitude': -90.06772}","[pickup, delivery]",$$,"{'address1': '411 Bourbon St', 'address2': '',...",+15044073717,(504) 407-3717,1554.302629
2,swVdc9piXvG0AXwL-F0GfQ,gallier-restaurant-and-oyster-bar-new-orleans,Gallier Restaurant & Oyster Bar,https://s3-media3.fl.yelpcdn.com/bphoto/IUq5PV...,False,https://www.yelp.com/biz/gallier-restaurant-an...,1318,"[{'alias': 'seafood', 'title': 'Seafood'}, {'a...",4.5,"{'latitude': 29.9533, 'longitude': -90.07077}",[delivery],$$,"{'address1': '129 Carondelet St', 'address2': ...",+15042675672,(504) 267-5672,1303.802022
3,6a4gLLFSgr-Q6CZXDLzBGQ,cochon-new-orleans-3,Cochon,https://s3-media2.fl.yelpcdn.com/bphoto/Z30o0o...,False,https://www.yelp.com/biz/cochon-new-orleans-3?...,4904,"[{'alias': 'cajun', 'title': 'Cajun/Creole'}, ...",4.0,"{'latitude': 29.94226, 'longitude': -90.06713}",[delivery],$$,"{'address1': '930 Tchoupitoulas St', 'address2...",+15045882123,(504) 588-2123,2209.984225
4,9qdUEbbjzjcpPbXKeRN4zw,nonno-s-cajun-cuisine-and-pastries-new-orleans,Nonno’s Cajun Cuisine and Pastries,https://s3-media3.fl.yelpcdn.com/bphoto/cG6oKn...,False,https://www.yelp.com/biz/nonno-s-cajun-cuisine...,199,"[{'alias': 'cajun', 'title': 'Cajun/Creole'}, ...",4.5,"{'latitude': 29.965278, 'longitude': -90.059175}","[pickup, delivery]",$$,"{'address1': '1940 Dauphine St', 'address2': N...",+15043541364,(504) 354-1364,2581.808622


,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
537,5QhbWM3l_NU9Zm-lTJPyMA,new-orleans-gulf-seafood-new-orleans,New Orleans Gulf Seafood,,False,https://www.yelp.com/biz/new-orleans-gulf-seaf...,1,"[{'alias': 'seafood', 'title': 'Seafood'}, {'a...",3.0,"{'latitude': 30.0156133621931, 'longitude': -8...",[],NaN,"{'address1': '9134 Chef Menteur Hwy', 'address...",+15042467329,(504) 246-7329,12068.203542
538,E48H5Zpw3ndlu7KLHGRyHg,ruths-chris-steak-house-metairie-2,Ruth's Chris Steak House,https://s3-media1.fl.yelpcdn.com/bphoto/lNnb-T...,False,https://www.yelp.com/biz/ruths-chris-steak-hou...,205,"[{'alias': 'steak', 'title': 'Steakhouses'}, {...",3.5,"{'latitude': 30.0043740015754, 'longitude': -9...","[pickup, delivery]",$$$$,"{'address1': '3633 Veterans Blvd', 'address2':...",+15048883600,(504) 888-3600,9352.935520
539,SyKHFYhRzW1hlwDga5EWWw,whole-foods-market-metairie,Whole Foods Market,https://s3-media2.fl.yelpcdn.com/bphoto/-yJyYr...,False,https://www.yelp.com/biz/whole-foods-market-me...,208,"[{'alias': 'grocery', 'title': 'Grocery'}, {'a...",4.0,"{'latitude': 30.00200103, 'longitude': -90.158...",[],$$,"{'address1': '3420 Veterans Memorial Blvd', 'a...",+15048888225,(504) 888-8225,8861.903837
540,qW-7X0NbZlGX7pvswoswCQ,la-maison-creole-harvey-3,La Maison Creole,https://s3-media2.fl.yelpcdn.com/bphoto/kYpSYQ...,False,https://www.yelp.com/biz/la-maison-creole-harv...,15,"[{'alias': 'catering', 'title': 'Caterers'}, {...",2.5,"{'latitude': 29.9063294, 'longitude': -90.0679...",[],NaN,"{'address1': '1605 8th St', 'address2': '', 'a...",+15043623908,(504) 362-3908,5739.500149
541,5KzQpZJOtT81nvDT9s1UbQ,china-house-new-orleans,China House,https://s3-media1.fl.yelpcdn.com/bphoto/TL_LyB...,False,https://www.yelp.com/biz/china-house-new-orlea...,17,"[{'alias': 'chinese', 'title': 'Chinese'}, {'a...",2.5,"{'latitude': 29.92527999774301, 'longitude': -...","[pickup, delivery]",$$,"{'address1': '3600 MacArthur Blvd', 'address2'...",+15043685520,(504) 368-5520,7074.404765


In [52]:
# check for duplicate ID's 
final_df.duplicated(subset='id').sum()

0